In [1]:
import os
import numpy as np
import open3d as o3d
import torch
import laspy

from laspy import LasData
from pathlib import Path
from tqdm import tqdm
from torch.cuda import amp
from torchsparse.utils.quantize import sparse_quantize
from torchsparse import SparseTensor

from panoramix3D.models import Panoramix3D
from panoramix3D.config import ModelConfig

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATA_DIR = Path.home() / 'Panoramix3D_data'
CONFIG_FILE = '../config/model/inference.yaml'
SAVE_CHUNKS = True
USE_STORED = True
SAVE_SEGMENTED = True
SAVE_ALL_SEGMENTS = True
CHUNK_SIZE = 15.0
MIN_POINTS_PER_PC = 2000
MIN_CHUNK_SIZE = 12.5
VOXEL_SIZE = 0.3

process_folder = DATA_DIR / 'to_process'
process_folder.mkdir(parents=True, exist_ok=True)

input_folder = process_folder / 'input'
output_folder = process_folder / 'output'
segmented_folder = process_folder / 'segmented'
segmented_by_folder = process_folder / 'segmented/by_folders'

input_folder.mkdir(parents=True, exist_ok=True)
output_folder.mkdir(parents=True, exist_ok=True)
segmented_folder.mkdir(parents=True, exist_ok=True)
segmented_by_folder.mkdir(parents=True, exist_ok=True)

In [3]:
def chunkerize(file: LasData, ext: str):
    points = file.points
    coords = np.vstack((file.x, file.y)).transpose()

    idx  = np.floor_divide(coords, CHUNK_SIZE).astype(int)
    idx -= idx.min(axis=0)

    idx = np.ravel_multi_index(idx.T, idx.max(axis=0) + 1)
    chunks = []
    for i, unique_idx in enumerate(tqdm(np.unique(idx))):
        chunk_points = points[idx == unique_idx]
        if len(chunk_points) < MIN_POINTS_PER_PC:
            continue
        
        min_x, max_x = chunk_points.x.min(), chunk_points.x.max()
        min_y, max_y = chunk_points.y.min(), chunk_points.y.max()
        if (max_x - min_x) < MIN_CHUNK_SIZE or (max_y - min_y) < MIN_CHUNK_SIZE:
            continue

        chunk_file = laspy.LasData(file.header)
        chunk_file.points = chunk_points
        chunks.append(chunk_file)
        
        if SAVE_CHUNKS:
            chunk_file.write(output_folder / f'plot_{i}{ext}')

    return chunks


In [5]:
extensions = ('.laz', '.las')
point_clouds = sorted(
            [f for f in input_folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions],
            key=lambda f: f.name
        )

config = ModelConfig.from_yaml(CONFIG_FILE)
model = Panoramix3D.from_config(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

pcd = o3d.geometry.PointCloud()
for point_cloud in point_clouds:
    if USE_STORED:
        chunks = [laspy.read(f) for f in output_folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions]
    else:
        chunks = chunkerize(laspy.read(point_cloud), point_cloud.suffix.lower())

    segmented_chunks = []

    instance_offset = 1
    for i, chunk in enumerate(tqdm(chunks)):
        coords = np.vstack((chunk.x, chunk.y, chunk.z)).transpose()
        min_coords = coords.min(axis=0)
        coords -= min_coords

        intensity = np.array(chunk.intensity)[:, None]
        min_intensity = np.min(intensity)
        max_intensity = np.max(intensity)
        i_norm = (intensity - min_intensity) / (max_intensity - min_intensity)

        voxels, indices, inverse_map = sparse_quantize(coords, VOXEL_SIZE, return_index=True, return_inverse=True)
        i_norm = i_norm[indices]

        min_x, max_x = chunk.x.min(), chunk.x.max()
        min_y, max_y = chunk.y.min(), chunk.y.max()

        if len(voxels) < 100:
            continue
        if (max_x - min_x) < MIN_CHUNK_SIZE or (max_y - min_y) < MIN_CHUNK_SIZE:
            continue
        if (voxels[:, 2].max() - voxels[:, 2].min()) < 20:
            continue

        voxels = torch.tensor(voxels, dtype=torch.int).to(device)
        batch_index = torch.zeros((voxels.shape[0], 1), dtype=torch.int, device=voxels.device)
        voxels = torch.cat([batch_index, voxels], dim=1)
        feat = torch.tensor(i_norm.astype(np.float32), dtype=torch.float).to(device)

        inputs = SparseTensor(coords=voxels, feats=feat)

        #print(f'Voxels shape: {voxels.shape}, Features shape: {feat.shape}')
        with amp.autocast(enabled=True):
            semantic_output_raw, specie_output_raw, _, _, _, instance_output_raw = model(inputs)

        voxels = semantic_output_raw.C.cpu().numpy()
        semantic_output = torch.argmax(semantic_output_raw.F.cpu(), dim=1).numpy()

        ng_mask = semantic_output != 0

        specie_output = np.zeros_like(semantic_output)
        specie_output[ng_mask] = torch.argmax(specie_output_raw.F.cpu(), dim=1).numpy() + 1

        instance_output_full = np.zeros_like(semantic_output)

        if instance_output_raw.F.shape[1] > 0:
            instance_output = torch.argmax(instance_output_raw.F.cpu(), dim=1).numpy()
            max_label = instance_output.max()
            instance_output_full[ng_mask] = instance_output + instance_offset
            instance_offset += max_label + 1

        semantic_output = semantic_output[inverse_map]
        specie_output = specie_output[inverse_map]
        instance_output = instance_output_full[inverse_map]

        out_file = laspy.LasData(header=chunk.header, points=chunk.points.copy())
        out_file.add_extra_dims([laspy.ExtraBytesParams(name="semantic_pred", type=np.int16), laspy.ExtraBytesParams(name="species_pred", type=np.int32), laspy.ExtraBytesParams(name="instance_pred", type=np.int32)])
        out_file.semantic_pred = semantic_output
        out_file.species_pred = specie_output
        out_file.instance_pred = instance_output

        segmented_chunks.append(out_file)
        if SAVE_ALL_SEGMENTS:
            out_file.write(segmented_by_folder / f'plot_{i}.las')

    with laspy.open(segmented_folder / 'combined.las', mode='w', header=segmented_chunks[0].header) as file:
        for chunk in segmented_chunks:
            file.write_points(chunk.points)


100%|██████████| 1001/1001 [00:24<00:00, 41.17it/s]
